In [1]:
from __future__ import print_function

from keras import backend as K
K.set_image_dim_ordering('th') # ensure our dimension notation matches

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras import utils
import numpy as np
from PIL import Image, ImageOps
import argparse
import math

import os
import os.path

import glob

Using TensorFlow backend.


In [2]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, output_dim=1024))
    model.add(Activation('tanh'))
    model.add(Dense(128*8*8))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Reshape((128, 8, 8), input_shape=(128*8*8,)))
    model.add(UpSampling2D(size=(4, 4)))
    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(4, 4)))
    model.add(Convolution2D(1, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))
    return model


def discriminator_model():
    model = Sequential()
    model.add(Convolution2D(
                        64, 5, 5,
                        border_mode='same',
                        input_shape=(1, 128, 128)))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(4, 4)))
    model.add(Convolution2D(128, 5, 5))
    model.add(Activation('tanh'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model


def generator_containing_discriminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model


def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

In [3]:
model = generator_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              103424    
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8192)              8396800   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8192)              32768     
_________________________________________________________________
activation_2 (Activation)    (None, 8192)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 8, 8)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 32, 32)       0         
__________

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (5, 5), padding="same")`
  del sys.path[0]


In [4]:
def load_data(pixels=128, verbose=False):
    print("Loading data")
    X_train = []
    paths = glob.glob(os.path.normpath(os.getcwd() + '/logos/*.jpg'))
    for path in paths:
        if verbose: print(path)
        im = Image.open(path)
        im = ImageOps.fit(im, (pixels, pixels), Image.ANTIALIAS)
        im = ImageOps.grayscale(im)
        #im.show()
        im = np.asarray(im)
        X_train.append(im)
    print("Finished loading data")
    return np.array(X_train)

def train(epochs, BATCH_SIZE, weights=False):
    """
    :param epochs: Train for this many epochs
    :param BATCH_SIZE: Size of minibatch
    :param weights: If True, load weights from file, otherwise train the model from scratch. 
    Use this if you have already saved state of the network and want to train it further.
    """
    X_train = load_data()
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
    discriminator = discriminator_model()
    generator = generator_model()
    if weights:
        generator.load_weights('goodgenerator.h5')
        discriminator.load_weights('gooddiscriminator.h5')
    discriminator_on_generator = \
        generator_containing_discriminator(generator, discriminator)
    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    discriminator_on_generator.compile(
        loss='binary_crossentropy', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)
    noise = np.zeros((BATCH_SIZE, 100))
    for epoch in range(epochs):
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = generator.predict(noise, verbose=0)
            #print(generated_images.shape)
            if index % 20 == 0 and epoch % 10 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                destpath = os.path.normpath(os.getcwd()+ "/logo-generated-images/"+str(epoch)+"_"+str(index)+".png")
                Image.fromarray(image.astype(np.uint8)).save(destpath)
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = discriminator.train_on_batch(X, y)
            print("batch %d d_loss : %f" % (index, d_loss))
            for i in range(BATCH_SIZE):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            discriminator.trainable = False
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * BATCH_SIZE)
            discriminator.trainable = True
            print("batch %d g_loss : %f" % (index, g_loss))
            if epoch % 10 == 9:
                generator.save_weights('goodgenerator.h5', True)
                discriminator.save_weights('gooddiscriminator.h5', True)

def clean(image):
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            if image[i][j] + image[i+1][j] + image[i][j+1] + image[i-1][j] + image[i][j-1] > 127 * 5:
                image[i][j] = 255
    return image
def generate(BATCH_SIZE):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('goodgenerator.h5')
    noise = np.zeros((BATCH_SIZE, 100))
    a = np.random.uniform(-1, 1, 100)
    b = np.random.uniform(-1, 1, 100)
    grad = (b - a) / BATCH_SIZE
    for i in range(BATCH_SIZE):
        noise[i, :] = np.random.uniform(-1, 1, 100)
    generated_images = generator.predict(noise, verbose=1)
    #image = combine_images(generated_images)
    print(generated_images.shape)
    for image in generated_images:
        image = image[0]
        image = image*127.5+127.5
        Image.fromarray(image.astype(np.uint8)).save("dirty.png")
        Image.fromarray(image.astype(np.uint8)).show()
        clean(image)
        image = Image.fromarray(image.astype(np.uint8))
        image.show()        
        image.save("clean.png")


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--mode", type=str)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--nice", dest="nice", action="store_true")
    parser.set_defaults(nice=False)
    args = parser.parse_args()
    return args


In [5]:
train(400, 10, False)

Loading data
Finished loading data
Epoch is 0
Number of batches 3


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(1, 128, 1..., padding="same")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5))`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (5, 5), padding="same")`
  del sys.path[0]


batch 0 d_loss : 0.625051
batch 0 g_loss : 0.699870
batch 1 d_loss : 0.598545
batch 1 g_loss : 0.685610
batch 2 d_loss : 0.559805
batch 2 g_loss : 0.688802
Epoch is 1
Number of batches 3
batch 0 d_loss : 0.539188
batch 0 g_loss : 0.686688
batch 1 d_loss : 0.500435
batch 1 g_loss : 0.700071
batch 2 d_loss : 0.472255
batch 2 g_loss : 0.705819
Epoch is 2
Number of batches 3
batch 0 d_loss : 0.460865
batch 0 g_loss : 0.701458
batch 1 d_loss : 0.432658
batch 1 g_loss : 0.699923
batch 2 d_loss : 0.408938
batch 2 g_loss : 0.709426
Epoch is 3
Number of batches 3
batch 0 d_loss : 0.410810
batch 0 g_loss : 0.696483
batch 1 d_loss : 0.403814
batch 1 g_loss : 0.698115
batch 2 d_loss : 0.409418
batch 2 g_loss : 0.703899
Epoch is 4
Number of batches 3
batch 0 d_loss : 0.413991
batch 0 g_loss : 0.683681
batch 1 d_loss : 0.418286
batch 1 g_loss : 0.681528
batch 2 d_loss : 0.424411
batch 2 g_loss : 0.697399
Epoch is 5
Number of batches 3
batch 0 d_loss : 0.422089
batch 0 g_loss : 0.661628
batch 1 d_los

batch 2 g_loss : 1.516988
Epoch is 44
Number of batches 3
batch 0 d_loss : 0.214623
batch 0 g_loss : 1.674325
batch 1 d_loss : 0.234204
batch 1 g_loss : 1.263367
batch 2 d_loss : 0.254914
batch 2 g_loss : 1.894930
Epoch is 45
Number of batches 3
batch 0 d_loss : 0.316669
batch 0 g_loss : 1.373129
batch 1 d_loss : 0.248516
batch 1 g_loss : 1.644613
batch 2 d_loss : 0.127281
batch 2 g_loss : 1.809843
Epoch is 46
Number of batches 3
batch 0 d_loss : 0.304113
batch 0 g_loss : 1.330969
batch 1 d_loss : 0.249917
batch 1 g_loss : 1.321899
batch 2 d_loss : 0.214623
batch 2 g_loss : 1.933152
Epoch is 47
Number of batches 3
batch 0 d_loss : 0.299843
batch 0 g_loss : 1.768464
batch 1 d_loss : 0.340470
batch 1 g_loss : 1.028545
batch 2 d_loss : 0.231616
batch 2 g_loss : 1.651897
Epoch is 48
Number of batches 3
batch 0 d_loss : 0.281073
batch 0 g_loss : 2.039861
batch 1 d_loss : 0.341879
batch 1 g_loss : 1.704404
batch 2 d_loss : 0.133553
batch 2 g_loss : 1.251820
Epoch is 49
Number of batches 3
ba

batch 1 d_loss : 0.385740
batch 1 g_loss : 1.114350
batch 2 d_loss : 0.234017
batch 2 g_loss : 1.458430
Epoch is 88
Number of batches 3
batch 0 d_loss : 0.498082
batch 0 g_loss : 1.372455
batch 1 d_loss : 0.304201
batch 1 g_loss : 1.326246
batch 2 d_loss : 0.312027
batch 2 g_loss : 1.504186
Epoch is 89
Number of batches 3
batch 0 d_loss : 0.442695
batch 0 g_loss : 1.412289
batch 1 d_loss : 0.380892
batch 1 g_loss : 1.101666
batch 2 d_loss : 0.296411
batch 2 g_loss : 1.404910
Epoch is 90
Number of batches 3
batch 0 d_loss : 0.442839
batch 0 g_loss : 1.243286
batch 1 d_loss : 0.518128
batch 1 g_loss : 1.719747
batch 2 d_loss : 0.252733
batch 2 g_loss : 1.483126
Epoch is 91
Number of batches 3
batch 0 d_loss : 0.452295
batch 0 g_loss : 1.393437
batch 1 d_loss : 0.440264
batch 1 g_loss : 1.334391
batch 2 d_loss : 0.334496
batch 2 g_loss : 1.871965
Epoch is 92
Number of batches 3
batch 0 d_loss : 0.495552
batch 0 g_loss : 1.442261
batch 1 d_loss : 0.367553
batch 1 g_loss : 1.255626
batch 2 

...

batch 2 d_loss : 0.399693
batch 2 g_loss : 1.861411
Epoch is 218
Number of batches 3
batch 0 d_loss : 0.359631
batch 0 g_loss : 1.783044
batch 1 d_loss : 0.381085
batch 1 g_loss : 1.843959
batch 2 d_loss : 0.340400
batch 2 g_loss : 1.814759
Epoch is 219
Number of batches 3
batch 0 d_loss : 0.349094
batch 0 g_loss : 1.615283
batch 1 d_loss : 0.369772
batch 1 g_loss : 2.185249
batch 2 d_loss : 0.335329
batch 2 g_loss : 1.881659
Epoch is 220
Number of batches 3
batch 0 d_loss : 0.485526
batch 0 g_loss : 1.977354
batch 1 d_loss : 0.402140
batch 1 g_loss : 1.717238
batch 2 d_loss : 0.244332
batch 2 g_loss : 1.217099
Epoch is 221
Number of batches 3
batch 0 d_loss : 0.404368
batch 0 g_loss : 1.505465
batch 1 d_loss : 0.256096
batch 1 g_loss : 1.099259
batch 2 d_loss : 0.362511
batch 2 g_loss : 2.429267
Epoch is 222
Number of batches 3
batch 0 d_loss : 0.355951
batch 0 g_loss : 1.810130
batch 1 d_loss : 0.376497
batch 1 g_loss : 1.951890
batch 2 d_loss : 0.266267
batch 2 g_loss : 1.522495
Epo

batch 0 d_loss : 0.221809
batch 0 g_loss : 2.273587
batch 1 d_loss : 0.185778
batch 1 g_loss : 1.573751
batch 2 d_loss : 0.185320
batch 2 g_loss : 1.676542
Epoch is 262
Number of batches 3
batch 0 d_loss : 0.250519
batch 0 g_loss : 2.130693
batch 1 d_loss : 0.243408
batch 1 g_loss : 1.824754
batch 2 d_loss : 0.383028
batch 2 g_loss : 3.012641
Epoch is 263
Number of batches 3
batch 0 d_loss : 0.310449
batch 0 g_loss : 1.587013
batch 1 d_loss : 0.230387
batch 1 g_loss : 1.552418
batch 2 d_loss : 0.211243
batch 2 g_loss : 2.257891
Epoch is 264
Number of batches 3
batch 0 d_loss : 0.295460
batch 0 g_loss : 2.328581
batch 1 d_loss : 0.251612
batch 1 g_loss : 2.151512
batch 2 d_loss : 0.214121
batch 2 g_loss : 2.004438
Epoch is 265
Number of batches 3
batch 0 d_loss : 0.154905
batch 0 g_loss : 1.669040
batch 1 d_loss : 0.308637
batch 1 g_loss : 2.873789
batch 2 d_loss : 0.312375
batch 2 g_loss : 2.376616
Epoch is 266
Number of batches 3
batch 0 d_loss : 0.202217
batch 0 g_loss : 1.842643
bat

batch 1 g_loss : 2.107994
batch 2 d_loss : 0.160424
batch 2 g_loss : 2.589324
Epoch is 305
Number of batches 3
batch 0 d_loss : 0.248084
batch 0 g_loss : 2.319132
batch 1 d_loss : 0.146888
batch 1 g_loss : 2.583828
batch 2 d_loss : 0.179294
batch 2 g_loss : 2.408369
Epoch is 306
Number of batches 3
batch 0 d_loss : 0.146995
batch 0 g_loss : 2.217721
batch 1 d_loss : 0.174683
batch 1 g_loss : 2.234214
batch 2 d_loss : 0.190567
batch 2 g_loss : 2.474622
Epoch is 307
Number of batches 3
batch 0 d_loss : 0.198784
batch 0 g_loss : 1.964261
batch 1 d_loss : 0.148541
batch 1 g_loss : 2.657916
batch 2 d_loss : 0.204992
batch 2 g_loss : 2.730995
Epoch is 308
Number of batches 3
batch 0 d_loss : 0.235043
batch 0 g_loss : 2.644003
batch 1 d_loss : 0.149702
batch 1 g_loss : 2.429887
batch 2 d_loss : 0.133572
batch 2 g_loss : 2.569873
Epoch is 309
Number of batches 3
batch 0 d_loss : 0.143705
batch 0 g_loss : 3.067023
batch 1 d_loss : 0.178862
batch 1 g_loss : 2.851830
batch 2 d_loss : 0.165027
bat

batch 0 d_loss : 0.142963
batch 0 g_loss : 2.530668
batch 1 d_loss : 0.146665
batch 1 g_loss : 3.057189
batch 2 d_loss : 0.153557
batch 2 g_loss : 3.576286
Epoch is 349
Number of batches 3
batch 0 d_loss : 0.127203
batch 0 g_loss : 2.803211
batch 1 d_loss : 0.120549
batch 1 g_loss : 3.483362
batch 2 d_loss : 0.113033
batch 2 g_loss : 2.690402
Epoch is 350
Number of batches 3
batch 0 d_loss : 0.123352
batch 0 g_loss : 2.692098
batch 1 d_loss : 0.169733
batch 1 g_loss : 4.111323
batch 2 d_loss : 0.127370
batch 2 g_loss : 2.648094
Epoch is 351
Number of batches 3
batch 0 d_loss : 0.148709
batch 0 g_loss : 2.843858
batch 1 d_loss : 0.208444
batch 1 g_loss : 3.960860
batch 2 d_loss : 0.147998
batch 2 g_loss : 2.584518
Epoch is 352
Number of batches 3
batch 0 d_loss : 0.152342
batch 0 g_loss : 2.480790
batch 1 d_loss : 0.148273
batch 1 g_loss : 2.690135
batch 2 d_loss : 0.143663
batch 2 g_loss : 3.408070
Epoch is 353
Number of batches 3
batch 0 d_loss : 0.183409
batch 0 g_loss : 3.163343
bat

batch 1 g_loss : 3.707450
batch 2 d_loss : 0.077444
batch 2 g_loss : 2.834242
Epoch is 392
Number of batches 3
batch 0 d_loss : 0.167573
batch 0 g_loss : 3.413344
batch 1 d_loss : 0.075425
batch 1 g_loss : 3.699921
batch 2 d_loss : 0.114394
batch 2 g_loss : 3.452021
Epoch is 393
Number of batches 3
batch 0 d_loss : 0.121394
batch 0 g_loss : 2.670513
batch 1 d_loss : 0.209962
batch 1 g_loss : 3.960820
batch 2 d_loss : 0.164387
batch 2 g_loss : 3.047945
Epoch is 394
Number of batches 3
batch 0 d_loss : 0.132839
batch 0 g_loss : 2.920641
batch 1 d_loss : 0.084748
batch 1 g_loss : 3.330678
batch 2 d_loss : 0.083101
batch 2 g_loss : 3.022659
Epoch is 395
Number of batches 3
batch 0 d_loss : 0.209323
batch 0 g_loss : 4.389356
batch 1 d_loss : 0.131559
batch 1 g_loss : 3.251202
batch 2 d_loss : 0.099528
batch 2 g_loss : 2.932229
Epoch is 396
Number of batches 3
batch 0 d_loss : 0.127110
batch 0 g_loss : 2.306334
batch 1 d_loss : 0.062618
batch 1 g_loss : 3.246945
batch 2 d_loss : 0.135499
bat

In [6]:
generate(1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=100, units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (5, 5), padding="same")`
  del sys.path[0]


1/1 [==============================] - 0s
(1, 1, 128, 128)
